# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Funçoes Auxiliares

In [ ]:
def pivot(A, b, k):
    n = len(b)
    
    max_index = np.argmax(np.abs(A[k:n, k])) + k
    
    if max_index != k:
        A[[k, max_index]] = A[[max_index, k]]
        b[[k, max_index]] = b[[max_index, k]]


In [ ]:
def back_substitution(U, y):
    m = len(y)
    x = np.zeros(m)
    
    for i in range(m - 1, -1, -1):
        x[i] = round((y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i], 3)
        
    return x


In [ ]:
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b)
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    return y


In [ ]:
def forward_substitution_LU(L, b, Pivot):
    m = len(b)
    y = np.zeros(m)
    
    for i in range(m):
        y[i] = b[Pivot[i] - 1] - np.dot(L[i, :i], y[:i])        
    return y


In [ ]:
def plot_convergence(iterations, errors):
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, errors, marker='o', linestyle='-')
    plt.xlabel('Iterações')
    plt.ylabel('Erro')
    plt.title('Convergência do Método Gauss-Jacobi')
    plt.yscale('log')
    plt.grid(True)
    plt.show()


# Métodos implementados: Gaussian Elimination, LU Decomposition, Cholesky Decomposition,Gauss Jacobi, Gauss Seidel, Newton

In [ ]:
def gauss_elimination(A, b, use_pivoting=False):
    n = len(b)
    A = A.astype(float)
    b = b.astype(float)

    for k in range(n-1):
        if use_pivoting:
            pivot(A, b, k)

        for i in range(k+1, n):
            m = round(A[i][k] / A[k][k], 3)
            A[i][k] = 0
            
            for j in range(k+1, n):
                A[i][j] = round(A[i][j] - m * A[k][j], 3)
                
            b[i] = round(b[i] - m * b[k], 3)

    return back_substitution(A, b)


In [ ]:
def LU_decomposition(A, b, use_pivoting=False):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    m, n = A.shape
    
    if m != n:
        raise ValueError("Matrix must be square")
    
    Pivot = np.arange(1, m + 1)
    PdU = 1.0
    Info = 0
    LU = A.copy()

    for j in range(n):
        if use_pivoting:
            pivot(LU, b, j)
        
        Pivot[j], Pivot[np.argmax(np.abs(LU[j:, j])) + j] = Pivot[np.argmax(np.abs(LU[j:, j])) + j], Pivot[j]

        if LU[j, j] == 0:
            if Info == 0:
                Info = j + 1
            continue

        PdU *= LU[j, j]

        if abs(LU[j, j]) != 0:
            r = 1 / LU[j, j]
            for i in range(j + 1, m):
                Mult = LU[i, j] * r
                LU[i, j] = Mult
                LU[i, j + 1:] -= Mult * LU[j, j + 1:]
        else:
            if Info == 0:
                Info = j + 1

    if Info != 0:
        raise ValueError(f"Matrix is singular at row {Info}")

    y = forward_substitution_LU(LU, b, Pivot)

    x = back_substitution(LU, y)

    return x, LU, Pivot, PdU, Info


In [ ]:
def cholesky_decomposition(A, b, use_pivoting=False):
    n = A.shape[0]
    L = np.zeros_like(A)
    Det = 1.0
    Info = 0
    
    if use_pivoting:
        for k in range(n):
            pivot(A, b, k)
    
    for j in range(n):
        Soma = 0.0
        for k in range(j):
            Soma += L[j, k] * L[j, k]
        
        t = A[j, j] - Soma
        if t > 0:
            L[j, j] = np.sqrt(t)
            r = 1 / L[j, j]
            Det *= t
        else:
            Info = j + 1
            raise ValueError("Matrix is not positive definite")
        
        for i in range(j + 1, n):
            Soma = 0.0
            for k in range(j):
                Soma += L[i, k] * L[j, k]
            L[i, j] = (A[i, j] - Soma) * r
    
    y = forward_substitution(L, b)
    x = back_substitution(L.T, y)
    
    return x, L, Det, Info


In [ ]:
def gauss_jacobi(matrixSize, A1, b1, toler, iterMax, x0=None):
    if x0 is None:
        x = np.zeros(matrixSize)
    else:
        x = x0
    x_new = np.zeros(matrixSize)
    iterations = []
    errors = []
    
    for k in range(iterMax):
        for i in range(matrixSize):
            sum_Ax = sum(A1[i][j] * x[j] for j in range(matrixSize) if i != j)
            x_new[i] = (b1[i] - sum_Ax) / A1[i][i]
        
        error = np.linalg.norm(x_new - x, ord=np.inf)
        iterations.append(k)
        errors.append(error)
        
        if error < toler:
            break
        
        x = x_new.copy()
    
    return x, iterations, errors

In [ ]:
def gauss_seidel(matrixSize, A1, b1, toler, iterMax, x0=None):
    if x0 is None:
        x = np.zeros(matrixSize)
    else:
        x = x0
    iterations = []
    errors = []
    
    for k in range(iterMax):
        x_old = x.copy()
        for i in range(matrixSize):
            sum_Ax1 = sum(A1[i][j] * x[j] for j in range(i))
            sum_Ax2 = sum(A1[i][j] * x_old[j] for j in range(i + 1, matrixSize))
            x[i] = (b1[i] - sum_Ax1 - sum_Ax2) / A1[i][i]
        
        error = np.linalg.norm(x - x_old, ord=np.inf)
        iterations.append(k)
        errors.append(error)
        
        if error < toler:
            break
    
    return x, iterations, errors

In [ ]:
def newton_method(F, J, x0, toler, iterMax):
    xk = x0
    
    for k in range(iterMax):
        Fxk = F(xk)
        Jxk = J(xk)
        
        if np.linalg.norm(Fxk) < toler:
            return xk
        
        sk = np.linalg.solve(Jxk, -Fxk)
        
        xk1 = xk + sk
        
        if np.linalg.norm(xk1 - xk) < toler:
            return xk1
        
        xk = xk1
    
    raise Exception("Newtons method did not converge.")


# Exercícios

## Exercício 1

In [ ]:
A1 = np.array([
    [0, 3, 2],
    [1, 4, 1],
    [0, 2, 5]])

b1 = np.array([5, 6, 7])
# 0x+3y+2z=5
# 1x+4y+1z=6
# 0x+2y+5z=7

A2 = np.array([
    [-2, -2, 0],
    [1, 3, -1],
    [0, -1, 2]])

b2 = np.array([-1, 3, 1])
# -2x+-2y+0z=-1
# 1x+3y+-1z=3
# 0x-1y+2z=1

A3 = np.array([
    [1, 2, 3],
    [2, 6, 0],
    [1, 0, 4]])

b3 = np.array([4, 8, 5])
# 1x+2y+3z=4
# 2x+6y+0z=8
# 1x+0y+4z=5


# -----------------------------------------------------------------------
# gauss_elimination
# -----------------------------------------------------------------------
# 1.a
print("------------ Gauss Elimination ------------")

solution = gauss_elimination(A1, b1, use_pivoting=True)
print(solution)

solution = gauss_elimination(A2, b2, use_pivoting=True)
print(solution)

solution = gauss_elimination(A3, b3, use_pivoting=True)
print(solution)

# -----------------------------------------------------------------------
# LU_decomposition
# -----------------------------------------------------------------------
# 1.b
print("\n------------ LU Decomposition ------------")

solution, LU, Pivot, PdU, Info = LU_decomposition(A1, b1, True)
print(solution)

solution, LU, Pivot, PdU, Info = LU_decomposition(A2, b2, True)
print(solution)
 
solution, LU, Pivot, PdU, Info = LU_decomposition(A3, b3, True)
print(solution)

# -----------------------------------------------------------------------
# cholesky_decomposition
# -----------------------------------------------------------------------
# 1.c

# print("\n------------ Cholesky Decomposition ------------")

solution, L, Det, Info = cholesky_decomposition(A1, b1, True)
print(solution)

solution, L, Det, Info = cholesky_decomposition(A2, b2, True)
print(solution)

solution, L, Det, Info = cholesky_decomposition(A3, b3, True)
print(solution)

# matrix is not symmetric matrix, so matrix is not positive definite

1.a<br>
Eliminação de Gauss<br>

Resultados Com Pivotamento Parcial<br>
[1. 1. 1.]<br>
[-1.5  2.   1.5]<br>
[ 5.8 -0.6 -0.2]<br>

Esses resultados indicam que o método conseguiu resolver os sistemas, ajustando as linhas para evitar divisões por zero ou números muito pequenos que poderiam causar instabilidade numérica.<br>

Resultados Sem Pivotamento Parcial<br>
[nan nan nan]<br>
[-1.5  2.   1.5]<br>
[ 5.8 -0.6 -0.2]<br>

O fato de termos [nan,nan,nan] no Sistema 1 sugere que houve um problema de divisão por zero ou uma situação onde a matriz perdeu sua estrutura triangular, o que levou a um resultado indefinido. Isso demonstra a importância do pivotamento parcial para garantir a estabilidade do algoritmo.<br>

<br>

1.b<br>
Fatoracao LU<br>

Resultados Com Pivotamento Parcial<br>
[1. 1. 1.]<br>
[-1.5  2.   1.5]<br>
[ 5.8 -0.6 -0.2]<br>

Os resultados são consistentes com os obtidos pela eliminação de Gauss com pivotamento, indicando que o processo de decomposição LU é equivalente em termos de resultados finais quando o pivotamento é utilizado.<br>

Resultados Sem Pivotamento Parcial<br>
[nan nan nan]<br>
[-1.5  2.   1.5]<br>
[-7.  3.  3.]<br>

Os resultados para o Sistema 3 são diferentes dos obtidos com pivotamento, mostrando uma potencial instabilidade ou erro devido à falta de pivotamento. Isso reforça a necessidade do pivotamento para obter resultados corretos.<br>

<br>

1.c<br>
Decomposição de Cholesky<br>

Para a decomposição de Cholesky tanto para tentativas com e sem pivotamento, os sistemas não puderam ser resolvidos pois a matriz não é simétrica. A decomposição de Cholesky requer que a matriz seja simétrica e positiva definida, condições que não são satisfeitas pelos sistemas fornecidos.<br>

## Exercício 2

In [ ]:
A1 = np.array([
    [1, 1, 3],
    [4, 1, 4],
    [5, 2, 1]])

b1 = np.array([-2, -3, 4])

# 1x+1y+3z=-2
# ax+1y+4z=-3
# 5x+2y+1z=4

# 2.a
print("\n------------ LU Decomposition ------------")
solution, LU, Pivot, PdU, Info = LU_decomposition(A1, b1, use_pivoting=True)
print(solution)

# 2.b
print("\n------------ Cholesky Decomposition ------------")
solution, L, Det, Info = cholesky_decomposition(A1, b1, use_pivoting=True)
print(solution)

# 2.c
print("\n------------ Gauss Elimination ------------")
solution = gauss_elimination(A1, b1, use_pivoting=True)
print(solution)

2.A<br>
LU Decomposition<br>
[ 1.  0. -1.]<br>

Para qualquer valor de α, podemos aplicar a eliminação de Gauss à matriz A. A decomposição  LU é viável desde que a matriz não tenha elementos nulos na posição dos pivôs durante a eliminação. Ou seja, a decomposição  LU não depende especificamente do valor de α se 𝛼 ≠ 0 e as operações de troca de linhas (caso seja necessário) são permitidas.

2.B<br>
Cholesky Decomposition<br>
ERRO: A matriz não é simétrica<br>

O sistema não pode ser resolvido com nenhum valor de 𝛼 por Fatoração Cholesky porque a matriz  A não é simétrica.

2.C<br>
Gauss Elimination<br>
[ 1.  0. -1.]<br>

## Exercício 3

In [ ]:
# Definição dos parâmetros
A = np.array([
    [17, -2, -3],
    [-5, 21, -2],
    [-5, -5, 22]
])

b = np.array([500, 200, 30])

# 17x – 2y – 3z = 500
# –5x + 21y – 2z = 200
# –5x – 5y + 22z = 30

matrixSize = A.shape[0]
toler = 0.0001
iterMax = 100


# 3.a
print("\n------------ Gauss Jacobi ------------")
x, iterations, errors = gauss_jacobi(matrixSize, A, b, toler, iterMax)
print(x)
# plot_convergence(iterations, errors)


print("\n------------ Gauss Seidel ------------")
x, iterations, errors = gauss_seidel(matrixSize, A, b, toler, iterMax)
print(x)
# plot_convergence(iterations, errors)


# 3.b
x0 = np.array([34, 19, 13])
iterMax = 2

print("Com valor inicial: [34, 19, 13] com 2 iteracoes:")
print("\n------------ Gauss Jacobi ------------")
x, iterations, errors = gauss_jacobi(matrixSize, A, b, toler, iterMax, x0)
print(x)
# plot_convergence(iterations, errors)


print("\n------------ Gauss Seidel ------------")
x, iterations, errors = gauss_seidel(matrixSize, A, b, toler, iterMax, x0)
print(x)
# plot_convergence(iterations, errors)


3.A<br>
Resultados:
Gauss Jacobi <br>
[33.99623653 18.89274677 13.38379396]<br>

Gauss Seidel <br>
[33.99631245 18.8928259  13.38389508] <br>

Comparando ambos os métodos, as soluções são bastante próximas, o que é esperado dado que ambos os métodos convergem para a mesma solução quando aplicados a sistemas diagonalmente dominantes. A diferença entre os resultados é bem pequena, o que indica que ambos os métodos estão convergindo de maneira eficaz para a solução do sistema.<br>

3.B<br>
Com valor inicial: [34, 19, 13] com 2 iterações:<br>
Gauss Jacobi <br>
[33.99656226 18.88209829 13.36325439]<br>

Gauss Seidel <br>
[33 18 12]<br>

Observa-se que os resultados obtidos com o ponto inicial especificado (34, 19, 13) e limitação de duas iterações apresentam algumas diferenças em relação aos resultados anteriores:<br>


A escolha da aproximação inicial pode influenciar a convergência e a precisão das soluções obtidas pelos métodos iterativos como Gauss-Jacobi e Gauss-Seidel. No exercício atual, a aproximação inicial [34,19,13] resultou em soluções finais que diferem ligeiramente daquelas obtidas com a aproximação inicial padrão. Isso ressalta a importância de experimentar diferentes aproximações iniciais para verificar a estabilidade e a convergência dos métodos iterativos.

Os resultados mostram que, ao modificar o ponto inicial e limitar o número de iterações, as soluções encontradas podem variar um pouco em comparação com os resultados obtidos com mais iterações ou sem restrição inicial. Isso é esperado, pois métodos iterativos como Gauss-Jacobi e Gauss-Seidel convergem para a solução, mas a rapidez e a trajetória da convergência podem depender da escolha do ponto inicial e do número de iterações permitidas.<br>


## Exercício 4

In [ ]:
A1 = np.array([
    [2, 5],
    [3, 1]])

b1 = np.array([-3, 2])

# 2x + 5y = -3
# 3x + y = 2

matrixSize = A.shape[0]
iterMax = 100
toler = 0.0001

print("\n------------ Gauss Jacobi ------------")
x, iterations, errors = gauss_jacobi(matrixSize, A1, b1, toler, iterMax)
print(x)
# plot_convergence(iterations, errors)


print("\n------------ Gauss Seidel ------------")
x, iterations, errors = gauss_seidel(matrixSize, A1, b1, toler, iterMax)
print(x)
# plot_convergence(iterations, errors)



# Permutacao das equacoes
A2 = np.array([
    [3, 1], 
    [2, 5]])

b2 = np.array([2, -3])

# 3x + 1y = 2
# 2x + 5y = -3

print("Com as equacoes permutadas:")

print("\n------------ Gauss Jacobi ------------")
x, iterations, errors = gauss_jacobi(matrixSize, A1, b1, toler, iterMax)
print(x)
# plot_convergence(iterations, errors)


print("\n------------ Gauss Seidel ------------")
x, iterations, errors = gauss_seidel(matrixSize, A1, b1, toler, iterMax)
print(x)
# plot_convergence(iterations, errors)


4.A<br>
Gauss-Jacobi: Este método iterativo não converge para o sistema dado. Os valores obtidos são extremamente grandes em magnitude, indicando divergência do método para resolver este sistema específico.<br>

Gauss-Seidel: O Gauss-Seidle também não converge. Os resultados são ainda maiores em magnitude, sugerindo uma divergência ainda mais .<br>

Os métodos de Gauss-Jacobi e Gauss-Seidel são sensíveis à convergência do sistema. No caso do sistema original, a matriz de coeficientes não apresenta uma estrutura que permita a convergência desses métodos, levando a soluções que crescem exponencialmente ao invés de convergir para uma solução.<br>

4.B<br>
Mesmo com a permutação das equações, os resultados dos métodos Gauss-Jacobi e Gauss-Seidel continuam sendo não convergentes e apresentam valores extremamente grandes.<br>

A permutação das equações não altera a natureza do problema em termos de convergência para os métodos iterativos. Os métodos de Gauss-Jacobi e Gauss-Seidel exigem que a matriz de coeficientes seja diagonalmente dominante ou que o sistema seja simétrico e definitivamente positivo para garantir a convergência. No caso desses sistemas, a falta de convergência sugere que as condições para aplicação desses métodos não foram atendidas.

## Exercício 5

In [ ]:
x0 = np.array([0.0, 0.0, 0.0])

def F(x):
    return np.array([
        3 * np.sin(x[0]) - 4 * x[1] - 12 * x[2] - 1,
        4 * x[0]**2 - 8 * x[1] - 10 * x[2] + 5,
        2 * np.exp(x[0]) + 2 * x[1] + 3 * x[2] - 8
    ])

def J(x):
    return np.array([
        [3 * np.cos(x[0]), -4, -12],
        [8 * x[0], -8, -10],
        [2 * np.exp(x[0]), 2, 3]
    ])
    
toler = 1e-6
iterMax = 100

solution = newton_method(F, J, x0, toler, iterMax)
print(solution)



5 <br>

Resultado obtido:<br>
[ 1.06999055  1.76139911 -0.45116738]<br>

A solução demonstra a eficácia do método de Newton para resolver sistemas de equações não lineares. A convergência para esses valores a partir da aproximação inicial (0,0,0) indica uma escolha apropriada e uma boa formulação do problema.